In [77]:
import csv
import pandas as pd
import numpy as np
import os
from binance.client import Client
import time
import re
import requests
import gdax
from datetime import datetime


In [250]:
api_input = open("APIKeyBin.txt", "r")
codes = api_input.readlines()
codes = [code.rstrip('\n') for code in codes]
api_key = codes[0]
api_secret = codes[1]

In [251]:
# Gather data, rename files as necessary
btc = pd.read_csv("BTCfills(12_21_17).csv")
eth = pd.read_csv("ETHfills(12_22_17).csv")
ltc = pd.read_csv("LTCfills(12_21_17).csv")
alts = pd.read_csv("OrderHistory.csv")

In [252]:
btc

,trade id,product,side,created at,size,size unit,price,fee,total,price/fee/total unit
0,23419416,BTC-USD,SELL,2017-11-08T17:28:41.965Z,0.059822,BTC,7800.00,0.000000,466.609260,USD
1,23500329,BTC-USD,BUY,2017-11-09T07:48:16.687Z,0.067610,BTC,7376.95,1.246883,-499.999964,USD
2,24375436,BTC-USD,SELL,2017-11-17T00:25:38.578Z,0.017521,BTC,7934.98,0.347578,138.683429,USD
3,25471432,BTC-USD,SELL,2017-11-30T01:54:35.492Z,0.033985,BTC,10484.00,0.000000,356.295071,USD
4,25852043,BTC-USD,BUY,2017-12-02T23:16:27.125Z,0.010000,BTC,11006.93,0.000000,-110.069300,USD
5,25852067,BTC-USD,SELL,2017-12-02T23:16:57.654Z,0.018604,BTC,11006.94,0.000000,204.778285,USD


In [253]:
btc_earnings = btc['total'].sum()

In [254]:
eth_earnings = eth['total'].sum()

In [255]:
ltc_earnings = ltc['total'].sum()

In [256]:
alts.head()

,Date,Pair,Type,Order Price,Order Amount,Avg Trading Price,Filled,Total,status
0,2017-12-21 05:55:06,MODBTC,SELL,0.0001562,102.21,0.0001563,102.210,0.015976,Filled
1,NaN,Date,Trading Price,Filled,Total,Fee,NaN,NaN,NaN
2,NaN,2017-12-21 05:55:06,0.0001563,102.21,0.01597542,0.00001598BTC,NaN,NaN,NaN
3,2017-12-20 08:56:43,XMRBTC,BUY,0.023801,0.692,0.023801,0.692,0.016471,Filled
4,NaN,Date,Trading Price,Filled,Total,Fee,NaN,NaN,NaN


In [257]:
alts_trans = alts.dropna(subset=['Date'])
alts_filled = alts_trans[alts_trans['status'] == 'Filled']
negated_sells = alts_filled.apply(lambda x: x.loc['Total'] if x.loc['Type'] == 'SELL' else -x.loc['Total'], axis=1).astype('float')
negated_coins = alts_filled.apply(lambda x: x.loc['Order Amount'] if x.loc['Type'] == 'SELL' else -float(x.loc['Order Amount']), axis=1).astype('float')
alts_filled.loc[:, 'Total'] = negated_sells
alts_filled.loc[:, 'Order Amount'] = negated_coins
buy_sell_amts = alts_filled.groupby(['Pair', 'Type'])[['Order Amount', 'Total']].sum()
buy_sell_amts

/Users/jacky/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


Order Amount        Total
Pair    Type                           
ARKBTC  BUY     -35.000000    -0.015558
        SELL     34.960000     0.012928
BTCUSDT BUY      -0.124588 -1841.370000
        SELL      0.131097  1843.210000
ETHBTC  BUY      -0.945000    -0.032604
IOTABTC BUY    -300.000000    -0.050137
        SELL    299.000000     0.050318
LTCBTC  BUY      -4.690000    -0.072435
        SELL     10.070000     0.165576
MODBTC  BUY    -152.320000    -0.027642
        SELL    152.160000     0.028004
QTUMBTC BUY      -7.000000    -0.011697
        SELL      6.990000     0.012589
TRXBTC  SELL    500.000000     0.000130
WTCBTC  BUY     -15.000000    -0.012646
        SELL     14.000000     0.009114
XMRBTC  BUY      -2.182000    -0.046423
        SELL      0.544000     0.013056

In [258]:
profits = buy_sell_amts.groupby('Pair').sum()
profits = profits.filter(regex='BTC$', axis = 0)
curr_btc_price = float(input("Current BTC price in USD: ").replace(',', ''))
profits_in_usd = pd.DataFrame()
profits_in_usd['Amount of Coin in Binance'] = profits['Order Amount']
profits_in_usd['Total'] = profits['Total'].apply(lambda x: x * curr_btc_price)

Current BTC price in USD: 13000


In [259]:
profits_in_usd

,Amount of Coin in Binance,Total
Pair,,
ARKBTC,-0.040,-34.18090
ETHBTC,-0.945,-423.85200
IOTABTC,-1.000,2.35521
LTCBTC,5.380,1210.83300
MODBTC,-0.160,4.70470
QTUMBTC,-0.010,11.59600
TRXBTC,500.000,1.69000
WTCBTC,-1.000,-45.92159
XMRBTC,-1.638,-433.77100


In [260]:
alts_earnings = profits_in_usd.sum()

In [261]:
alts_earnings + btc_earnings + eth_earnings + ltc_earnings

Amount of Coin in Binance    874.466211
Total                        667.332631
dtype: float64

In [262]:
btc_earnings

556.2967804399215

In [263]:
eth_earnings

-314.509168702054

In [264]:
ltc_earnings

132.09159926903092

In [148]:
client = Client(api_key, api_secret)

In [184]:
client.get_withdraw_history()

{'success': True,
 'withdrawList': [{'address': 'LYainifoMdCf6jBs9YNKv3Snd2xuQd66Gd',
   'addressTag': '',
   'amount': 3.21936549,
   'applyTime': 1513230251000,
   'asset': 'LTC',
   'id': '7b31b7df79054d6c8ece5e1e2046d80d',
   'status': 6,
   'successTime': 1513231679000,
   'txId': '4f358f3eb327bb9d8a725b4cafbdea802a379c373f2738e8ba261f3d06533636'},
  {'address': '0x27B5905687F744dF40B3fE86c947397008115cC4',
   'addressTag': '',
   'amount': 0.944055,
   'applyTime': 1513088648000,
   'asset': 'ETH',
   'id': '66ea26ce15634e35969c9b5466feaaa7',
   'status': 6,
   'successTime': 1513089521000,
   'txId': '0x3899e30d5dad37b5a3403720b1124d09418874feb899621f10bf40afee8ed450'},
  {'address': 'LWzNdGy3dQ7oPZU7NQzwaDWZYZ433K8ycj',
   'addressTag': '',
   'amount': 2.41531,
   'applyTime': 1513085525000,
   'asset': 'LTC',
   'id': '63de3b8a1b2e402db5d764460f71059a',
   'status': 6,
   'successTime': 1513089150000,
   'txId': 'ba9b280d9d21d8df83902f08bb52b5ab6cfabeb9e132e67b04d683dd058771a

In [190]:
# get all trades from all symbols

all_symbols = client.get_all_tickers()
all_symbols = [coin['symbol'] for coin in all_symbols]

trades = []
for symbol in all_symbols:
    this_coins_trade = client.get_my_trades(symbol = symbol)
    for trade in this_coins_trade:
        trade['symbol'] = symbol
    trades.extend(this_coins_trade)
    time.sleep(0.1)


In [223]:
trades_df = pd.DataFrame(trades)
trades_df['coinOfInterest'] = trades_df['symbol'].apply(lambda x: re.sub(r'BTC$|USDT$', '', x))
coin_of_basis = trades_df.apply(lambda x: x['symbol'].replace(x['coinOfInterest'], ''), axis=1)
trades_df['coinOfBasis'] = coin_of_basis
trades_df

,commission,commissionAsset,id,isBestMatch,isBuyer,isMaker,orderId,price,qty,symbol,time,coinOfInterest,coinOfBasis
0,0.00046700,ETH,6158916,True,True,True,26691743,0.03450100,0.46700000,ETHBTC,1513088433121,ETH,BTC
1,0.00008000,ETH,6158917,True,True,True,26691743,0.03450100,0.08000000,ETHBTC,1513088433234,ETH,BTC
2,0.00008000,ETH,6158918,True,True,True,26691743,0.03450100,0.08000000,ETHBTC,1513088433333,ETH,BTC
3,0.00004100,ETH,6158919,True,True,True,26691743,0.03450100,0.04100000,ETHBTC,1513088433661,ETH,BTC
4,0.00015000,ETH,6158920,True,True,True,26691743,0.03450100,0.15000000,ETHBTC,1513088433721,ETH,BTC
5,0.00012700,ETH,6158921,True,True,True,26691743,0.03450100,0.12700000,ETHBTC,1513088434079,ETH,BTC
6,0.00001601,BTC,1815340,True,False,False,11156093,0.01469000,1.09000000,LTCBTC,1513076134922,LTC,BTC
7,0.00005450,BTC,1815344,True,False,True,11156093,0.01469000,3.71000000,LTCBTC,1513076135898,LTC,BTC
8,0.00228000,LTC,1815665,True,True,True,11156932,0.01459200,2.28000000,LTCBTC,1513076332906,LTC,BTC
9,0.00002449,BTC,1830042,True,False,True,11186707,0.01749500,1.40000000,LTCBTC,1513081852082,LTC,BTC


In [78]:
api_input = open("APIKeyGDAX.txt", "r")
codes = api_input.readlines()
codes = [code.rstrip('\n') for code in codes]
GDAXapi_key = codes[0]
GDAXapi_secret = codes[1]
GDAXpass = codes[2]

In [79]:
auth_client = gdax.AuthenticatedClient(GDAXapi_key, GDAXapi_secret, GDAXpass)

In [81]:
request = auth_client.get_accounts()

In [82]:
holdings = pd.DataFrame(request)
holdings

,available,balance,currency,hold,id,profile_id
0,1421.2792110068971,2026.2792110068971000,USD,605.0000000000000000,498953be-01aa-4e2f-b942-645159b365ed,df592a89-8873-4436-a5d9-4d8410c8fdc0
1,0.5,0.5000000000000000,LTC,0.0000000000000000,519dad36-ad08-4f45-b41a-800330a92be8,df592a89-8873-4436-a5d9-4d8410c8fdc0
2,1.60274544,1.6027454400000000,ETH,0.0000000000000000,a41fd816-0d38-486d-9d39-8e520a6bf7f7,df592a89-8873-4436-a5d9-4d8410c8fdc0
3,0,0.0000000000000000,BTC,0.0000000000000000,f8abc92c-acd1-44cc-859a-e99328f2f63e,df592a89-8873-4436-a5d9-4d8410c8fdc0
4,0,0.0000000000000000,BCH,0.0000000000000000,12ed5862-38a9-45d5-89fd-d0486e09e7a1,df592a89-8873-4436-a5d9-4d8410c8fdc0


In [83]:
profile_ids = {account['currency']: account['id'] for account in request}

In [84]:
auth_client.get_account_history(profile_ids['LTC'])

[[{'amount': '1.0000000000000000',
   'balance': '1.5000000000000000',
   'created_at': '2017-12-26T05:24:38.268715Z',
   'details': {'order_id': '90fdfa1c-f7a9-4733-9677-a591d7579c71',
    'product_id': 'LTC-USD',
    'trade_id': '20902470'},
   'id': 580750792,
   'type': 'match'},
  {'amount': '0.5000000000000000',
   'balance': '0.5000000000000000',
   'created_at': '2017-12-21T05:21:57.836593Z',
   'details': {'order_id': 'c026765a-4da1-4c2c-b9ea-a95e4ca2a16c',
    'product_id': 'LTC-USD',
    'trade_id': '19919431'},
   'id': 551486901,
   'type': 'match'},
  {'amount': '-0.9183654900000000',
   'balance': '0.0000000000000000',
   'created_at': '2017-12-21T04:39:03.360988Z',
   'details': {'order_id': 'ef61ac38-cea6-4328-9da7-3dfd72124abe',
    'product_id': 'LTC-USD',
    'trade_id': '19906618'},
   'id': 551259960,
   'type': 'match'},
  {'amount': '-2.3000000000000000',
   'balance': '0.9183654900000000',
   'created_at': '2017-12-15T15:25:48.359079Z',
   'details': {'order_id

In [85]:
fills = auth_client.get_fills()

In [86]:
fills_df = pd.DataFrame(fills[0])
fills_df['created_at'] = fills_df['created_at'].apply(lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%S.%fZ'))
fills_df = fills_df.sort_values(by='created_at').reset_index(drop=True)

In [87]:
fills_df

,created_at,fee,liquidity,order_id,price,product_id,profile_id,settled,side,size,trade_id,usd_volume,user_id
0,2017-11-08 17:28:41.965,0.0000000000000000,M,46ae039a-2436-4e35-b974-90e810884db9,7800.00000000,BTC-USD,df592a89-8873-4436-a5d9-4d8410c8fdc0,True,sell,0.05982170,23419416,None,59b38107aebae700d82d9ffa
1,2017-11-09 07:48:16.687,1.2468827033425000,T,24bbe1fb-203b-40ae-b840-e5d67489f70f,7376.95000000,BTC-USD,df592a89-8873-4436-a5d9-4d8410c8fdc0,True,buy,0.06760966,23500329,None,59b38107aebae700d82d9ffa
2,2017-11-17 00:25:38.578,0.3475775159360000,T,8b7b5df1-407c-42b5-9333-d183b806f8af,7934.98000000,BTC-USD,df592a89-8873-4436-a5d9-4d8410c8fdc0,True,sell,0.01752128,24375436,139.0310063744000000,59b38107aebae700d82d9ffa
3,2017-11-20 01:40:06.755,3.0943461943944000,T,2d40eafd-1c85-4ce3-b473-96f041133395,72.27000000,LTC-USD,df592a89-8873-4436-a5d9-4d8410c8fdc0,True,sell,14.27215624,11532041,1031.4487314648000000,59b38107aebae700d82d9ffa
4,2017-11-20 01:40:06.755,0.0037464768000000,T,2d40eafd-1c85-4ce3-b473-96f041133395,72.27000000,LTC-USD,df592a89-8873-4436-a5d9-4d8410c8fdc0,True,sell,0.01728000,11532042,1.2488256000000000,59b38107aebae700d82d9ffa
5,2017-11-20 01:40:06.755,1.3459166194815000,T,2d40eafd-1c85-4ce3-b473-96f041133395,72.27000000,LTC-USD,df592a89-8873-4436-a5d9-4d8410c8fdc0,True,sell,6.20781615,11532043,448.6388731605000000,59b38107aebae700d82d9ffa
6,2017-11-23 23:06:52.861,2.9910269176956000,T,8f2b91da-b5e2-4372-bae1-dd3fa2af3a11,74.86000000,LTC-USD,df592a89-8873-4436-a5d9-4d8410c8fdc0,True,buy,13.31831382,12015571,997.0089725652000000,59b38107aebae700d82d9ffa
7,2017-11-24 00:22:37.864,2.9666544034050000,T,d03683b0-7676-4ad8-92e9-9b6a44e7b5ea,74.25000000,LTC-USD,df592a89-8873-4436-a5d9-4d8410c8fdc0,True,sell,13.31831382,12020535,988.8848011350000000,59b38107aebae700d82d9ffa
8,2017-11-24 00:27:09.686,0.0000000000000000,M,b315ce82-a380-47ab-8e4e-4d45476a04bd,74.20000000,LTC-USD,df592a89-8873-4436-a5d9-4d8410c8fdc0,True,buy,14.00000000,12020667,1038.8000000000000000,59b38107aebae700d82d9ffa
9,2017-11-24 00:41:54.536,0.0299102680782000,T,d5f838f6-b122-406c-a393-3a5c0d9cd5a3,73.62000000,LTC-USD,df592a89-8873-4436-a5d9-4d8410c8fdc0,True,buy,0.13542637,12021353,9.9700893594000000,59b38107aebae700d82d9ffa


In [320]:
def get_average_price(table):
    #checks
    table = table.sort_values(by='created_at').reset_index(drop=True)
    unique_coins = table['product_id'].unique()
    unique_coins = [str(coin) for coin in unique_coins]
    if 'nan' in unique_coins and len(unique_coins) != 2:
        print("More than one currency in average price check")
        return
    elif 'nan' not in unique_coins and len(unique_coins) != 1:
        print("More than one currency in average price check")
        return
    start_index = 0
    table['price'] = table['price'].astype(float)
    table['size'] = table['size'].astype(float)
    if 'rolling_average' in table.columns:
        index = table['rolling_average'].index[table['rolling_average'].apply(np.isnan)]
        table_index = table.index.values.tolist()
        start_index = table_index.index(index[0])
    if start_index == 0:
        table['rolling_average'] = pd.Series(table.loc[0, 'price'])
        table['rolling_stash'] = pd.Series(table.loc[0, 'size'])
        start_index += 1
    curr_price_index = start_index
    while curr_price_index < len(table):
        if table['type'][curr_price_index] == 'buy' or table['type'][curr_price_index] == 'deposit':
            prev_price = table['rolling_average'][curr_price_index-1]
            prev_stash = table['rolling_stash'][curr_price_index-1]
            this_price = table['price'][curr_price_index]
            this_stash = table['size'][curr_price_index]
            total_stash = prev_stash + this_stash
            table.loc[curr_price_index, 'rolling_average'] = (prev_price*(prev_stash/total_stash)) + (this_price*(this_stash/total_stash))
            table.loc[curr_price_index, 'rolling_stash'] = total_stash
        else:
            remove_stash = table['size'][curr_price_index]
            table.loc[curr_price_index, 'rolling_average'] = table['rolling_average'][curr_price_index-1]
            table.loc[curr_price_index, 'rolling_stash'] = table['rolling_stash'][curr_price_index-1] + remove_stash
        curr_price_index += 1
    return table

In [204]:
def create_fills_table(coin, fills):
    fills_df = pd.DataFrame(fills[0])
    fills_df.loc[:, 'created_at'] = fills_df['created_at'].apply(lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%S.%fZ'))
    fills_df = fills_df.sort_values(by='created_at').reset_index(drop=True)
    product_id = coin + '-USD'
    coin_chart = fills_df[fills_df['product_id'] == product_id].reset_index(drop=True)
    coin_chart.rename(columns = {'side':'type'}, inplace = True)
    coin_chart['size'] = coin_chart['size'].astype(float)
    negated_sells = coin_chart.apply(lambda x: -x.loc['size'] if x.loc['type'] == 'sell' else x.loc['size'], axis=1).astype('float')
    coin_chart['size'] = negated_sells
    return coin_chart

In [317]:
def create_transfers_table(coin):
    coin_hist = auth_client.get_account_history(profile_ids[coin])
    coin_hist = pd.DataFrame(coin_hist[0])
    coin_transfers = coin_hist[coin_hist['type'] == 'transfer']
    coin_transfers.loc[:, 'created_at'] = coin_transfers['created_at'].apply(lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%S.%fZ'))
    coin_transfers.loc[:, 'transfer_id'] = coin_transfers['details'].apply(lambda x: x.get('transfer_id'))
    coin_transfers.rename(columns = {'amount': 'size'}, inplace=True)
    coin_transfers.loc[:, 'id'] = coin_transfers['id'].astype(str)
    coin_transfers.loc[:, 'size'] = coin_transfers['size'].astype(float)
    coin_transfers = coin_transfers.reset_index(drop=True)
    prices = []
    types = []
    for index, row in coin_transfers.iterrows():
        if row['size'] < 0:
            cost = float(input("What is the loss (in USD) of the {0} {1} for the withdrawal at {2}".format(-row['size'], coin, row['created_at'])))
            price = cost/(-row['size'])
            prices.append(price)
            types.append('withdrawal')
        else:
            cost = float(input("What is the gain (in USD) of the {0} {1} for the deposit at {2}".format(row['size'], coin, row['created_at'])))
            price = cost/row['size']
            prices.append(price)
            types.append('deposit')
    prices_series = pd.Series(prices)
    types_series = pd.Series(types)
    coin_transfers.loc[:, 'price'] = prices_series
    coin_transfers.loc[:, 'type'] = types_series
    return coin_transfers

In [206]:
def append_transfers(transfers, main_table):
    transfers = transfers.loc[:, ['size', 'created_at', 'id', 'type', 'transfer_id', 'price']]
    main = pd.concat([main_table, transfers], ignore_index=True)
    return main

In [231]:
def drop_nonessential(table):
    columns_to_drop = set(['order_id', 'id', 'trade_id', 'user_id', 'liquidity', 'profile_id', 'transfer_id'])
    columns_exist = columns_to_drop.intersection(table.columns)
    if list(columns_exist) != []:
        dropped_table = table.drop(list(columns_exist), axis=1)
    return dropped_table

In [331]:
def get_gains(table):
    sells_withdraws = table[(table['type'] == 'sell') | (table['type'] == 'withdraw')]
    price_diff = (sells_withdraws['price'] - sells_withdraws['rolling_average'])*np.negative(sells_withdraws['size'])
    return price_diff.sum()

In [321]:
BTC_transfers = create_transfers_table('BTC')
BTC_fills = create_fills_table('BTC', fills)
BTC_table = append_transfers(BTC_transfers, BTC_fills)
BTC_table = BTC_table.sort_values(by='created_at').reset_index(drop=True)
BTC_avg = get_average_price(BTC_table)

/Users/jacky/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/Users/jacky/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:337: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Users/jacky/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:2746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/p

What is the gain (in USD) of the 0.00860447 BTC for the deposit at 2017-12-02 23:08:49.62883494.56
What is the gain (in USD) of the 0.03398465 BTC for the deposit at 2017-11-30 01:53:55.549125351.53
What is the gain (in USD) of the 0.01752128 BTC for the deposit at 2017-11-16 23:43:01.850298137.19
What is the loss (in USD) of the 0.06760966 BTC for the withdrawal at 2017-11-09 07:57:18.148940499.20
What is the gain (in USD) of the 0.0598217 BTC for the deposit at 2017-09-16 05:38:08.130303234.73


In [334]:
LTC_transfers = create_transfers_table('LTC')
LTC_fills = create_fills_table('LTC', fills)
LTC_table = append_transfers(LTC_transfers, LTC_fills)
LTC_table = LTC_table.sort_values(by='created_at').reset_index(drop=True)
LTC_avg = get_average_price(LTC_table)

/Users/jacky/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/Users/jacky/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:337: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Users/jacky/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:2746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/p

What is the gain (in USD) of the 3.21836549 LTC for the deposit at 2017-12-14 06:24:27.3902151016.10
What is the loss (in USD) of the 6.21936549 LTC for the withdrawal at 2017-12-14 04:06:15.2740971959.10
What is the gain (in USD) of the 2.41431 LTC for the deposit at 2017-12-12 14:44:14.110648913.71
What is the loss (in USD) of the 4.8 LTC for the withdrawal at 2017-12-12 10:18:31.2233021188.09
What is the loss (in USD) of the 2.0 LTC for the withdrawal at 2017-12-12 09:53:49.121355502.72
What is the gain (in USD) of the 17.53564497 LTC for the deposit at 2017-11-20 01:39:10.4336581267.82
What is the gain (in USD) of the 2.96160742 LTC for the deposit at 2017-09-16 05:39:19.277842169.52


In [337]:
ETH_transfers = create_transfers_table('ETH')
ETH_fills = create_fills_table('ETH', fills)
ETH_table = append_transfers(ETH_transfers, ETH_fills)
ETH_table = ETH_table.sort_values(by='created_at').reset_index(drop=True)
ETH_avg = get_average_price(ETH_table)

/Users/jacky/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/Users/jacky/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:337: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Users/jacky/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:2746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/p

What is the gain (in USD) of the 0.934055 ETH for the deposit at 2017-12-12 14:49:01.301000549.36
What is the gain (in USD) of the 0.42434979 ETH for the deposit at 2017-09-16 05:38:51.368638116.22


In [335]:
drop_nonessential(LTC_avg)

,created_at,fee,price,product_id,settled,size,type,usd_volume,rolling_average,rolling_stash
0,2017-09-16 05:39:19.277842,NaN,57.239187,NaN,NaN,2.961607,deposit,NaN,57.239187,2.961607e+00
1,2017-11-20 01:39:10.433658,NaN,72.299593,NaN,NaN,17.535645,deposit,NaN,70.123545,2.049725e+01
2,2017-11-20 01:40:06.755000,3.0943461943944000,72.270000,LTC-USD,True,-14.272156,sell,1031.4487314648000000,70.123545,6.225096e+00
3,2017-11-20 01:40:06.755000,1.3459166194815000,72.270000,LTC-USD,True,-6.207816,sell,448.6388731605000000,70.123545,1.728000e-02
4,2017-11-20 01:40:06.755000,0.0037464768000000,72.270000,LTC-USD,True,-0.017280,sell,1.2488256000000000,70.123545,4.059253e-16
5,2017-11-23 23:06:52.861000,2.9910269176956000,74.860000,LTC-USD,True,13.318314,buy,997.0089725652000000,74.860000,1.331831e+01
6,2017-11-24 00:22:37.864000,2.9666544034050000,74.250000,LTC-USD,True,-13.318314,sell,988.8848011350000000,74.860000,0.000000e+00
7,2017-11-24 00:27:09.686000,0.0000000000000000,74.200000,LTC-USD,True,14.000000,buy,1038.8000000000000000,74.200000,1.400000e+01
8,2017-11-24 00:41:54.536000,0.0299102680782000,73.620000,LTC-USD,True,0.135426,buy,9.9700893594000000,74.194443,1.413543e+01
9,2017-11-24 00:42:02.902000,0.2748387118554000,73.620000,LTC-USD,True,1.244402,buy,91.6129039518000000,74.147964,1.537983e+01


In [332]:
get_gains(BTC_avg)

238.6343219962

In [336]:
get_gains(LTC_avg)

673.581715628598

In [338]:
get_gains(ETH_avg)

767.3386054800826